In [1]:
pip install -q "tensorflow-text==2.8.*"

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

2022-04-03 01:02:47.359385: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-03 01:02:47.362270: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
embeddings_index = {}
with open('glove.6B.50d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [7]:
#Open data
data_file = open('Data.txt').read()
data_file

'The important thing is not to stop questioning. Curiosity has its own reason for existence. One cannot help but be in awe when he contemplates the mysteries of eternity, of life, of the marvelous structure of reality. It is enough if one tries merely to comprehend a little of this mystery each day.'

In [8]:
#Create tokenizer object
tokenizer = Tokenizer()

#Convert data to lowercase
data = data_file.lower().split('.')

In [9]:
#Create dictionary of words with the frequency they occur
#Every word gets unique value > 0
#0 is reserved for padding
tokenizer.fit_on_texts(data)

#Transforms sentences into set of integers from the dictionary
input_sequences = tokenizer.texts_to_sequences(data)

#Counts total words
total_words = len(tokenizer.word_index) + 1

#Prints it all out
print(tokenizer.word_index, '\n')
print(input_sequences, '\n')
print(total_words, '\n')

{'of': 1, 'the': 2, 'is': 3, 'to': 4, 'one': 5, 'important': 6, 'thing': 7, 'not': 8, 'stop': 9, 'questioning': 10, 'curiosity': 11, 'has': 12, 'its': 13, 'own': 14, 'reason': 15, 'for': 16, 'existence': 17, 'cannot': 18, 'help': 19, 'but': 20, 'be': 21, 'in': 22, 'awe': 23, 'when': 24, 'he': 25, 'contemplates': 26, 'mysteries': 27, 'eternity': 28, 'life': 29, 'marvelous': 30, 'structure': 31, 'reality': 32, 'it': 33, 'enough': 34, 'if': 35, 'tries': 36, 'merely': 37, 'comprehend': 38, 'a': 39, 'little': 40, 'this': 41, 'mystery': 42, 'each': 43, 'day': 44} 

[[2, 6, 7, 3, 8, 4, 9, 10], [11, 12, 13, 14, 15, 16, 17], [5, 18, 19, 20, 21, 22, 23, 24, 25, 26, 2, 27, 1, 28, 1, 29, 1, 2, 30, 31, 1, 32], [33, 3, 34, 35, 5, 36, 37, 4, 38, 39, 40, 1, 41, 42, 43, 44], []] 

45 



In [10]:
#Pad sequences
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen = 20, padding ='post', value = 0)
input_sequences

array([[ 2,  6,  7,  3,  8,  4,  9, 10,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [11, 12, 13, 14, 15, 16, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [19, 20, 21, 22, 23, 24, 25, 26,  2, 27,  1, 28,  1, 29,  1,  2,
        30, 31,  1, 32],
       [33,  3, 34, 35,  5, 36, 37,  4, 38, 39, 40,  1, 41, 42, 43, 44,
         0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]], dtype=int32)

In [11]:
input_sequences = np.array(input_sequences)
print(input_sequences.shape)
print(input_sequences)

(5, 20)
[[ 2  6  7  3  8  4  9 10  0  0  0  0  0  0  0  0  0  0  0  0]
 [11 12 13 14 15 16 17  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [19 20 21 22 23 24 25 26  2 27  1 28  1 29  1  2 30 31  1 32]
 [33  3 34 35  5 36 37  4 38 39 40  1 41 42 43 44  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


In [12]:
#Shift array by one to create targets 
output_sequences = np.array(np.roll(input_sequences, 80))
output_sequences[-1] = 0
print(output_sequences.shape)
print(output_sequences)

(5, 20)
[[11 12 13 14 15 16 17  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [19 20 21 22 23 24 25 26  2 27  1 28  1 29  1  2 30 31  1 32]
 [33  3 34 35  5 36 37  4 38 39 40  1 41 42 43 44  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


In [13]:
input_shape = input_sequences.shape[1:]
input_shape

(20,)

In [14]:
output_shape = output_sequences.shape[1:]
output_shape

(20,)

In [15]:
input_sequences = tf.convert_to_tensor(input_sequences)
output_sequences = tf.convert_to_tensor(output_sequences)

2022-04-03 01:11:22.405207: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-03 01:11:22.405486: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-03 01:11:22.405670: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-gdm2z): /proc/driver/nvidia/version does not exist
2022-04-03 01:11:22.408257: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
#prepare embedding matrix
word_index = dict(zip(tokenizer.word_index, range(len(tokenizer.word_index))))
num_tokens = len(tokenizer.word_index) + 2
embedding_dim = 50
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 44 words (0 misses)


In [25]:
pretrained_embedding_layer = keras.layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [27]:
#Convert to embedding vectors
#embedding_model = keras.Sequential()

input = keras.layers.Input(shape = input_sequences.shape)

word_input = keras.layers.Input(shape = input_sequences.shape[1])

embedding_layer = keras.layers.Embedding(input_dim = total_words, output_dim = 512, mask_zero = True, input_length = input_sequences.shape[1])(word_input) 

word_vectors = keras.layers.Flatten()(embedding_layer)

embedding_model = keras.Model(word_input, word_vectors)

embedding_model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(), metrics=[keras.metrics.SparseCategoricalAccuracy()])
                    
keras.utils.plot_model(embedding_model,to_file='model.png', show_shapes=True,expand_nested=True)

word_vectors = embedding_model.predict(input_sequences)
word_vectors

array([[ 8.1657991e-03, -4.3009698e-02, -1.2794066e-02, ...,
         4.0928017e-02, -2.0601345e-02, -1.2529600e-02],
       [ 1.6522799e-02,  1.5623178e-02, -1.7693747e-02, ...,
         4.0928017e-02, -2.0601345e-02, -1.2529600e-02],
       [-4.2396512e-02,  4.2234566e-02,  3.9753143e-02, ...,
         1.7466400e-02,  9.9800527e-05,  3.3993125e-03],
       [ 4.2923320e-02, -2.9993057e-02, -2.7239729e-02, ...,
         4.0928017e-02, -2.0601345e-02, -1.2529600e-02],
       [ 3.9652433e-02,  4.9618069e-02,  3.1441823e-03, ...,
         4.0928017e-02, -2.0601345e-02, -1.2529600e-02]], dtype=float32)

In [ ]:
#Calculate positional encoding
